In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm, colors
import glob

from numba import njit, vectorize, float32

from typing import Callable, Optional, Union

import hoomd
import gsd.hoomd

import sys
import time
import pickle
import gc
import pathlib
import os

import signac

from scipy import optimize

In [6]:
from monk import nb, prep, pair, render, utils, grid, workflow
import freud

parent = pathlib.Path(os.getcwd()).parent / "config.yaml"
config = workflow.get_config(parent.as_posix())

# mpl.rcParams["text.usetex"] = True
# mpl.rcParams["text.latex.preamble"].append(r'\usepackage{xfrac}')

In [7]:
project: signac.Project = signac.get_project(root=config['root'])
project.doc

{'avail_seed': 11, 'dt': 0.005, 'step_unit': 1000, 'temp_steps': 20, 'equil_time': 40, 'run_time': 10, 'max_alpha_time': 1000, 'alpha_iters': 10, '_status': {}}

In [8]:
for i, job in enumerate(project):

    print(i, job, job.doc)

0 a03bdc2a5be70a1e699a3f825fb8877b {'init': True, 'seed': 0, 'init_temp': 1.5, 'temp_steps': [0.07500000000000001, 0.15000000000000002, 0.22500000000000003, 0.30000000000000004, 0.375, 0.45000000000000007, 0.525, 0.6000000000000001, 0.675, 0.75, 0.8250000000000001, 0.9000000000000001, 0.9750000000000001, 1.05, 1.125, 1.2000000000000002, 1.2750000000000001, 1.35, 1.425, 1.5], 'primary_equilibration_done': True, 'sparse_secondary_equilibration_done': True, 'secondary_equilibration_done': True, 'fine_quench_done': True, 'fire_applied': True}
1 1961d54ebec38ae1cebeabd263ddba5b {'init': True, 'seed': 2, 'init_temp': 1.5, 'temp_steps': [0.07500000000000001, 0.15000000000000002, 0.22500000000000003, 0.30000000000000004, 0.375, 0.45000000000000007, 0.525, 0.6000000000000001, 0.675, 0.75, 0.8250000000000001, 0.9000000000000001, 0.9750000000000001, 1.05, 1.125, 1.2000000000000002, 1.2750000000000001, 1.35, 1.425, 1.5], 'primary_equilibration_done': True, 'fine_quench_done': True, 'secondary_equi

In [9]:
# files = glob.glob(job.fn("analysis/long_fire_temp*.gsd"))
files = glob.glob(job.fn("fine/equil_temp*.gsd"))
files

['/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.300.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.562.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.487.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.450.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.525.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.675.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.637.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.337.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.375.gsd',
 '/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equ

In [10]:
traj = gsd.hoomd.open(files[-2])

In [11]:
(traj[2].configuration.step - traj[1].configuration.step)*0.005

5.0

In [15]:
proj_doc = project.doc
proj_doc["step_unit"]//50*proj_doc["dt"]

0.1

In [13]:
len(traj)

171

In [64]:
proj_doc["dt"]

0.005